# Scheduling execution of tasks with QCG-PilotJob library

In this Notebook we present how to use the QCG-PilotJob library to schedule and execute set of tasks on pre-allocated resources. Those resources might be an allocation created in scheduling system but can be also (mostly for the tests) local defined resources.

## Installing QCG-PilotJob

Before we start our examples, we need to install QCG-PilotJob module. The easiest way is to install directly from GitHub.

In [ ]:
!pip install qcg-pilotjob

# Scenario

In following examples we want to run some computations and visualize results. In the first step we took the colloid example for *Lammps* package, where we simulate collisions of big particles with the small ones. This simulation produces a bunch of images presenting the simulation progress. The second step is to prepare the images for the rendering a movie - we have to rename the files by removing the trailing zeros from the names. In the following step we will render a movie based on the images - we will use the *ffmpeg* application for this. The last step will be removing the images produced by the simulation.

# Batch mode

QCG-PilotJob allows to schedule and execute jobs in batch mode. In this mode we prepare the description of jobs we want to run, along with their resource requirements and relationships as an input file in JSON format and run the QCG-PilotJob Manager service (`qcg-pm-service` command) with this file as an argument. The sample file containing description of our scenario we put in the *lammps.json* file. Below we present the contents of this file.

In [ ]:
!cat examples/lammps.json

The **jobs** list in the **submit** request contains 4 elements, each representing job we want to run. As we can see each job has similar structure. We define the:
* execution parameters (**execution** element) which contains name of the program to execute (**exec** element, with program arguments - the **args** list) or commands to execute (**script** element), the files where to/from redirect **stdin**, **stdout** and **stderr** streams and working directory (**wd** element) where to run job - if working directory contains relative path it will always be relative to the working directory of QCG PilotJob Manager
* dependencies parameters (**dependencies** element) where **after** list contain list of jobs which must finish (correctly, with exit code 0) before the job start
The second, control request, **finishAfterAllTasksDone** tells QCG-PilotJob Manager to stop listening for new requests after all submitted jobs finish.

## Run service in batch mode

The only required parameter for running QCG-PilotJob Manager is **--file-path** which should point to the file with jobs description. The rest of the used parameters are responsible for:
* **--wd** - points the working directory where output and temporary files should be written, all relative job's working directories will be created as subdirectory in the service' working directory
* **--show-progress** - print the progress of executing jobs
The execution of the following command should took less than minute.

To make sure the working directory is not present we can delete it

In [ ]:
!rm -fR lammps-example-out

We are ready to execute our jobs

In [ ]:
!qcg-pm-service --wd lammps-example-out --show-progress --file-path examples/lammps.json

Now the output movie should be generated as **colloid.mp4** file in the lammps-example-out/lammps directory. We can display it with following command.

In [ ]:
from IPython.display import Video
Video("lammps-example-out/lammps/colloid.mp4")

# API mode

The QCG PilotJob Manager contains API which allows to communicate with service during runtime. In the client program we can connect to already running QCG PilotJob Manager service instance, or we can create such instance. The second approach is more convinient, so in the following examples we will use it. To initialize API we create *LocalManager* object which will launch the QCG PilotJob Manager service in background thread, allowing us to communicate with it. 

In [ ]:
from qcg.pilotjob.api.manager import LocalManager

m = LocalManager()

## Avilable resources

Now we can for example request service to get available resources.

In [ ]:
print(m.resources())

## Submitting jobs
To submit a jobs, we need to create a list with jobs we want to submit

In [ ]:
from qcg.pilotjob.api.job import Jobs
jobs = Jobs()

Now we can define jobs we want to submit, either as keyword arguments:

In [ ]:
jobs.add(name='first', exec='date', stdout='first.out')

or by passing the dictionary with arguments:

In [ ]:
jobs.add({'name': 'second', 'script': 'echo "the second job"', 'stdout': 'second.out'})

Now we can submit a job list to be executed

In [ ]:
ids = m.submit(jobs)

In return we will got the identifier of submitted jobs (we don't have to define job identifiers in jobs list, if they are not present they will be defined automatically)

In [ ]:
print(ids)

We can ask QCG PilotJob Manager about status of jobs - passing the identifiers of the jobs to the **info** method.

In [ ]:
info = m.info(ids, withChilds=True)
print(info)

In return we will got a dictionary containing the current status of our jobs along with the additional data such as:
* the allocation of resources for each job (**runtime**/**allocation**)
* the timing of starting and finishing jobs (**runtime**/**rtime**, **runtime**/**history**)

To get already parsed information we can invoke the **info_parsed** method, which will:
* decode runtime into timedelta data type
* parse allocation into separate nodes
* parse history

In [ ]:
pinfo = m.info_parsed(ids)
for job_name, job_info in pinfo.items():
    print('job name: {}'.format(job_name))
    print('total cores: {}'.format(job_info.total_cores))
    print('run time: {}'.format(job_info.time))

## Iteration jobs

The QCG-PilotJob Manager can also run a bunch of jobs which has the same execution program and resource requirements. To use this feature we need to specify the **iteration** element in job description with either the number of iterations or a range of program iterations. In the following example we will run 10 instances of program.

In [ ]:
ids = m.submit(Jobs().add(name='iter_job2', exec='/bin/echo', stdout='echo.out.$it', args=['$it'], iteration=10))
m.wait4(ids)

In the iteration job descriptions we can use **$it** variable which will be replaced in a final description with the iteration identifier. In this example we have used this identifier to both to generate a name postfix for the output files and to define the parametrised input argument.  

As we can see, the set of  output files **echo.out.*** has been created and each of them should contain the proper iteration number in a postfix.

In order to get basic information about the iteration job you can use **info** in the same way as for the regular jobs:

In [ ]:
print(m.info(ids))

In case of iteration jobs, the output of **info** method contains also information about iteration jobs: the range of iterations, the total number of iterations, the number of finished and failed jobs. 

If we need detailed information about iterations we can request such information using the **info_parsed** method with **withChilds** parameter, for example:

In [ ]:
pinfo = m.info_parsed(ids, withChilds=True)
for job_name, job_info in pinfo.items():
    for child in job_info.childs:
        print(child)

## Parallel jobs

Until now, our jobs used only single core to execute. Defining a job we can specify how many nodes and cores are required to execute each job. The number of nodes and cores can be specified as exact number, or as range, where QCG-PilotJob Manager service will try to allocate as much resources (in the specified range) as currently are available. To ilustrate parallel execution we will use a simple MPI application that will print information about number of instances launched. The code is available in the *examples/mpi/mpi_print.c* file.

In [ ]:
!cat examples/mpi/mpi_print.c

Next, let's compile it

In [ ]:
!mpicc -Wall -o examples/mpi/mpi_print examples/mpi/mpi_print.c

So, lets submit this code on a single core, and wait for it to finish:

In [ ]:
mpi_single_ids = m.submit(Jobs().add(name='mpi_single', exec='mpirun', args=['-n', '${ncores}', 'examples/mpi/mpi_print'], stdout='mpi_single.out'))
m.wait4(mpi_single_ids)
mpi_single_info = m.info_parsed(mpi_single_ids)['mpi_single']
print('mpi_single total cores: {}'.format(mpi_single_info.total_cores))

and check the output file:

In [ ]:
!cat mpi_single.out

Now, let's submit a parallel job, but this time we pass the **numCores** keyword argument with exact number of required cores:

In [ ]:
mpi_parallel_ids = m.submit(Jobs().add(name='mpi_parallel', exec='mpirun', args=['-n', '${ncores}', 'examples/mpi/mpi_print'], \
                                       stdout='mpi_parallel.out', numCores=2))
m.wait4(mpi_parallel_ids)
mpi_parallel_info = m.info_parsed(mpi_parallel_ids)['mpi_parallel']
print('mpi_parallel total cores: {}'.format(mpi_parallel_info.total_cores))

and check the output file:

In [ ]:
!cat mpi_parallel.out

> It's worth to mention that mpirun (in it's standard configuration, outside of scheduling system) without **-n** parameter will run as many instances of the program as all available cores in the system.

## Cleanup

Using the QCG-PilotJob API we should always properly finish manager instance with the **finish** method which will send the finish message to the manager service and wait (in case of **LocalManager**) for the manager's service thread finish.

In [ ]:
m.finish()

## Resources
The QCG-PilogJob Manager schedules and executes jobs on available resources. The information about resources can be obtained automatically, so without any resource related arguments, the library will check the environment to look for variables set by Slurm scheduling system. In case where those variables exists, the information about Slurm allocation's resources will be obtained. In other cases the QCG PilogJob Manager will execute job's on all locally available cores (the number of nodes will be set to 1). We can change the number of locally available resources, by using `-nodes` parameter during invocation. 

In [ ]:
from qcg.pilotjob.api.manager import LocalManager

m = LocalManager(['--nodes', 8, '--log', 'error'])
print(m.resources())

As we can see the QCG PilotJob Manager returned this time eight available cores, the same as we requested. We have to remember that these are actually *virtual* cores, and it means that QCG PilotJob Manager will execute in the same time eight jobs, but physically this jobs will share the physically cores so their execution will be slower. This mode is only for testing purposes.

In [ ]:
m.finish()